Makine öğrenimi, bir trade-offs problemidir. Klasik sorun, overfittinge karşı underfitting'dir. Overfitting, bir model eğitim verilerine çok iyi uyduğunda ve iyi genelleme yapamadığında gerçekleşir. Underfitting ise tam tersidir: model, verilerdeki kalıpları bulmak için çok basittir.

Karar ağaçları, makine öğreniminde son derece popüler ve kullanışlı bir modeldir. Ama kolayca overfit olabilir. Budama, overfiti önlemek/üstesinden gelmek için en çok kullanılan tekniklerden biridir. Bu notebookta, yaygın olarak kullanılan 2 budama türünü tartışacağız.

<br>

***1. Prepruning*** <br>
***2. Postpruning***

<br>

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day7-DecisionTree/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART1/Day7-DecisionTree/notebooks/"

DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
df = pd.read_csv(DATASET_PATH + 'heart.csv')
df.head()

Bu veri setinin temel ayrıntılarına girmiyoruz. Bu notebook'un temel amacı, karar ağacının nasıl ön budama (pre-prunning) ve son budama (post-prunning) yapılacağını size göstermektir.

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

print(X.shape)
print(y.shape)

Eğitim ve Test Kümelerinin Ayrılması

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,stratify=y)
print(x_train.shape)
print(x_test.shape)

İlk önce herhangi bir şey yapmadan normal bir karar ağacını oluşturacağız ve sonuçları kontrol edeceğiz.

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=0)
clf.fit(x_train,y_train)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)


### Karar Ağacının Görselleştirilmesi

In [ ]:
plt.figure(figsize=(20,20))
features = df.columns
classes = ['Not heart disease','heart disease']
tree.plot_tree(clf,feature_names=features,class_names=classes,filled=True)
plt.show()

In [ ]:
# helper function
def plot_confusionmatrix(y_train_pred,y_train,dom):
    print(f'{dom} Confusion matrix')
    cf = confusion_matrix(y_train_pred,y_train)
    sns.heatmap(cf,annot=True,yticklabels=classes
               ,xticklabels=classes,cmap='Blues', fmt='g')
    plt.tight_layout()
    plt.show()
    

In [ ]:
print(f'Train score {accuracy_score(y_train_pred,y_train)}')
print(f'Test score {accuracy_score(y_test_pred,y_test)}')
plot_confusionmatrix(y_train_pred,y_train,dom='Train')
plot_confusionmatrix(y_test_pred,y_test,dom='Test')

Eğitim verilerimizde %100 doğruluğa sahip olduğumuzu görebiliriz.

Ancak test veri kümesinde iyi genelleme yapmıyor. Sadece %75 doğruluğumuz var.

Buradan görüldüğü üzere model açıkça overfitting olmuş durumda. Budama yoluyla bunu önlemeye çalışacağız.

## 1. Ön budama teknikleri (Pre pruning techniques)

Ön budama (Pre pruning), karar ağacının büyümesini erken bir aşamada durdurmaktan başka bir şey değildir. Bunun için kısıtlamalar koyarak ağaçların büyümesini sınırlayabiliriz. **max_depth** , **min_samples** vb. gibi parametreleri sınırlayabiliriz.

Bunu yapmanın etkili bir yolu, bu parametreler üzerinde grid search yapmak ve test verilerinde daha iyi performans sağlayan optimum değerleri seçmemizdir.


Kontrol edeceğimiz parametreler :

* max_depth: karar ağacının maksimum derinliği
* min_sample_split: Bir dahili düğümü bölmek için gereken minimum örnek sayısı.
* min_samples_leaf: Bir yaprak düğümünde olması gereken minimum örnek sayısı.


In [ ]:
params = {'max_depth': [2,4,6,8,10,12],
         'min_samples_split': [2,3,4],
         'min_samples_leaf': [1,2]}

clf = tree.DecisionTreeClassifier()
gcv = GridSearchCV(estimator=clf,param_grid=params)
gcv.fit(x_train,y_train)


In [ ]:
model = gcv.best_estimator_
model.fit(x_train,y_train)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print(f'Train score {accuracy_score(y_train_pred,y_train)}')
print(f'Test score {accuracy_score(y_test_pred,y_test)}')
plot_confusionmatrix(y_train_pred,y_train,dom='Train')
plot_confusionmatrix(y_test_pred,y_test,dom='Test')

In [ ]:
plt.figure(figsize=(20,20))
features = df.columns
classes = ['Not heart disease','heart disease']
tree.plot_tree(model,feature_names=features,class_names=classes,filled=True)
plt.show()

Ağacın budandığını ve test doğruluğunda iyileşme olduğunu görebiliyoruz. Ancak yine de daha iyi bir iyileştirmeye ihtiyaç duyuyor.

## 2. Son budama teknikleri (Post pruning techniques)

Birkaç post budama tekniği vardır. Maliyet karmaşıklığı budaması (Cost complexity pruning) bunların en önemlilerinden biridir.

## Maliyet karmaşıklığı budaması (Cost Complexity Pruning)

Karar ağaçları kolayca overfit olabilir. Bundan kaçınmanın bir yolu, kısıtlamalar koyarak ağaçların büyümesini sınırlamaktır. **max_depth** , **min_samples** vb. gibi parametreleri sınırlayabiliriz. **Ancak en etkili yol, maliyet karmaşıklığı budaması gibi son budama yöntemlerini kullanmaktır.** Bu, test doğruluğunu artırmaya ve daha iyi bir model elde etmeye yardımcı olur.

Minimum maliyetli karmaşıklık budaması, "en zayıf bağlantıya" sahip düğümü özyinelemeli olarak bulur. En zayıf halka, en küçük etkili alfaya sahip düğümlerin ilk önce budandığı etkin bir alfa ile karakterize edilir.

Maliyet karmaşıklığı budaması tamamen **alfa** için doğru parametreyi bulmakla ilgilidir. Bu ağaç için alfa değerlerini alacağız ve budanan ağaçlarla doğruluğunu kontrol edeceğiz.




In [ ]:
path = clf.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(ccp_alphas)

In [ ]:
# For each alpha we will append our model to a list
clfs = []
for ccp_alpha in ccp_alphas:
    clf = tree.DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(x_train, y_train)
    clfs.append(clf)

> Clfs ve ccp_alphas içindeki son elemanı kaldıracağız, çünkü bu sadece bir düğümü olan önemsiz bir ağaçtır.



In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]
node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
plt.scatter(ccp_alphas,node_counts)
plt.scatter(ccp_alphas,depth)
plt.plot(ccp_alphas,node_counts,label='no of nodes',drawstyle="steps-post")
plt.plot(ccp_alphas,depth,label='depth',drawstyle="steps-post")
plt.legend()
plt.show()

Gözlem: Alfa arttıkça düğüm sayısı ve derinlik azalır



In [ ]:
train_acc = []
test_acc = []
for c in clfs:
    y_train_pred = c.predict(x_train)
    y_test_pred = c.predict(x_test)
    train_acc.append(accuracy_score(y_train_pred,y_train))
    test_acc.append(accuracy_score(y_test_pred,y_test))

plt.scatter(ccp_alphas,train_acc)
plt.scatter(ccp_alphas,test_acc)
plt.plot(ccp_alphas,train_acc,label='train_accuracy',drawstyle="steps-post")
plt.plot(ccp_alphas,test_acc,label='test_accuracy',drawstyle="steps-post")
plt.legend()
plt.title('Accuracy vs alpha')
plt.show()

**Alpha seçimi = 0.020**

In [ ]:
clf_ = tree.DecisionTreeClassifier(random_state=0,ccp_alpha=0.020)
clf_.fit(x_train,y_train)
y_train_pred = clf_.predict(x_train)
y_test_pred = clf_.predict(x_test)

print(f'Train score {accuracy_score(y_train_pred,y_train)}')
print(f'Test score {accuracy_score(y_test_pred,y_test)}')
plot_confusionmatrix(y_train_pred,y_train,dom='Train')
plot_confusionmatrix(y_test_pred,y_test,dom='Test')

Artık modelimizin overfit olmadığını ve test verilerindeki performansın arttığını görebiliyoruz.

In [ ]:
plt.figure(figsize=(20,20))
features = df.columns
classes = ['Not heart disease','heart disease']
tree.plot_tree(clf_,feature_names=features,class_names=classes,filled=True)
plt.show()

Karar ağacının boyutunun önemli ölçüde azaldığını görebiliriz. Ayrıca son-budamanın, ön-budamadan çok daha etkili olduğunu gördük.

Not: Bu çekirdekte metrik olarak doğruluğu kullandık. Ancak hedef etiket dengesiz. Bu nedenle auc, f1 puanı vb. gibi metrikleri kullanmak daha iyidir.